In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, r2_score

In [15]:
EmailData = pd.read_csv('/content/drive/MyDrive/ML Assignments/41149_B2/emails.csv')
EmailDataCopy = EmailData.copy()
EmailDataCopy.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [16]:
EmailDataCopy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Columns: 3002 entries, Email No. to Prediction
dtypes: int64(3001), object(1)
memory usage: 118.5+ MB


In [17]:
EmailDataCopy.describe()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
count,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,...,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000
mean,6.640565,6.188128,5.143852,3.075599,3.124710,2.627030,55.517401,2.466551,2.024362,10.600155,...,0.005027,0.012568,0.010634,0.098028,0.004254,0.006574,0.004060,0.914733,0.006961,0.290023
std,11.745009,9.534576,14.101142,6.045970,4.680522,6.229845,87.574172,4.314444,6.967878,19.281892,...,0.105788,0.199682,0.116693,0.569532,0.096252,0.138908,0.072145,2.780203,0.098086,0.453817
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,3.000000,1.000000,1.000000,2.000000,1.000000,28.000000,1.000000,0.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,7.000000,4.000000,3.000000,4.000000,2.000000,62.250000,3.000000,1.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,210.000000,132.000000,344.000000,89.000000,47.000000,77.000000,1898.000000,70.000000,167.000000,223.000000,...,4.000000,7.000000,2.000000,12.000000,3.000000,4.000000,3.000000,114.000000,4.000000,1.000000


In [18]:
EmailDataCopy.isna().sum()

Email No.     0
the           0
to            0
ect           0
and           0
             ..
military      0
allowing      0
ff            0
dry           0
Prediction    0
Length: 3002, dtype: int64

In [21]:
SignificantColumns = []
for word in EmailDataCopy.columns:
  if word not in ENGLISH_STOP_WORDS:
    SignificantColumns.append(word)
EmailDataCopy = EmailDataCopy[SignificantColumns]
EmailDataCopy.head()

,Email No.,ect,hou,enron,s,com,gas,deal,meter,hpl,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,1,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,24,27,1,95,3,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,1,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,22,10,0,36,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,17,9,0,19,0,2,0,3,0,...,0,0,0,0,0,0,0,1,0,0


In [22]:
x = EmailDataCopy.drop(['Email No.', 'Prediction'], axis = 1)
x.head()

,ect,hou,enron,s,com,gas,deal,meter,hpl,e,...,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry
0,1,0,0,3,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
1,24,27,1,95,3,1,0,0,0,141,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,2,0,2,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,22,10,0,36,0,0,2,1,0,79,...,0,0,0,0,0,0,0,0,0,0
4,17,9,0,19,0,2,0,3,0,71,...,0,0,0,0,0,0,0,0,1,0


In [23]:
y = EmailDataCopy['Prediction']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Prediction, dtype: int64

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 1)

In [25]:
KNNModel = KNeighborsClassifier(n_neighbors = 8)
KNNModel.fit(x_train, y_train)
y_pred = KNNModel.predict(x_test)
pd.DataFrame({'Actual':y_test, 'Preds':y_pred})

,Actual,Preds
2408,0,0
2730,1,1
2936,0,0
847,0,0
1785,1,1
...,...,...
769,0,1
3091,0,0
1387,1,1
2831,1,1


In [26]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('MAE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 Score:', r2_score(y_test, y_pred))
print('Accuracy Score:', accuracy_score(y_test, y_pred))

MAE: 0.1500386697602475
MSE: 0.1500386697602475
MAE: 0.3873482538494881
R2 Score: 0.28137032453245325
Accuracy Score: 0.8499613302397525


In [27]:
SVCModel = SVC(C = 1.0, kernel = 'rbf', gamma = 'auto')
SVCModel.fit(x_train, y_train)
y_pred = SVCModel.predict(x_test)
pd.DataFrame({'Actual':y_test, 'Preds':y_pred})

,Actual,Preds
2408,0,0
2730,1,1
2936,0,0
847,0,0
1785,1,1
...,...,...
769,0,0
3091,0,0
1387,1,1
2831,1,1


In [28]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('MAE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 Score:', r2_score(y_test, y_pred))
print('Accuracy Score:', accuracy_score(y_test, y_pred))

MAE: 0.10208816705336426
MSE: 0.10208816705336426
MAE: 0.31951238951465444
R2 Score: 0.5110354785478548
Accuracy Score: 0.8979118329466357
